In [1]:
from product.stock_category import StockCategory
from datetime import datetime, timedelta
import pytz
import pandas as pd
from tqdm import tqdm
import numpy as np

## Currently not modeling stock category due to low actual accuracy

In [2]:
params = {}
product = StockCategory(params)
product.subscribe()
sim = product.create_sim()
# product.load()
# # product.transformed

stock_category sim:  13%|████████████▊                                                                                     | 66/505 [00:06<00:38, 11.32it/s]

'date'


stock_category sim:  16%|███████████████▋                                                                                  | 81/505 [00:07<00:35, 11.81it/s]

'date'


stock_category sim: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 505/505 [00:48<00:00, 10.42it/s]

'date'
'date'
'date'
'date'


In [3]:
# ## ai code
# year = 2018
# params = {"training_years":4}
# training_years = 5
# transformed = product.initial_transform()
# prediction_set = product.create_prediction_set(transformed,year,training_years,1)
# training_set = product.create_training_set(transformed,year,training_years,1)